In [2]:
run run_residuals -l -o 1

431 File aspcapStar-r5-v603-2M03403073+2429143.fits does not appear to exist on the server ...
432 File aspcapStar-r5-v603-2M03405126+2335544.fits does not appear to exist on the server ...
433 File aspcapStar-r5-v603-2M03413958+2345471.fits does not appear to exist on the server ...
434 File aspcapStar-r5-v603-2M03415366+2327287.fits does not appear to exist on the server ...
435 File aspcapStar-r5-v603-2M03415868+2342263.fits does not appear to exist on the server ...
436 File aspcapStar-r5-v603-2M03420383+2442454.fits does not appear to exist on the server ...
437 File aspcapStar-r5-v603-2M03422154+2439527.fits does not appear to exist on the server ...
438 File aspcapStar-r5-v603-2M03422760+2502492.fits does not appear to exist on the server ...
439 File aspcapStar-r5-v603-2M03424189+2411583.fits does not appear to exist on the server ...
440 File aspcapStar-r5-v603-2M03432662+2459395.fits does not appear to exist on the server ...
441 File aspcapStar-r5-v603-2M03433660+2327141.fit

ValueError: could not broadcast input array from shape (24) into shape (484)